In [1]:
from queue import Queue
import torchvision
import inspect 
import typing
import requests

In [71]:
def scraping_lib(thing): 
    # thing = torchvision.transforms.functional

    q = Queue()
    allowlist = {}
    # print(f'thing {thing}')
    q.put(thing)

    empty_typing_hints = list()
    modules_list = list()
    classes_list = list()
    while(not q.empty()):

        module = q.get()
        # print(f'Processing {module.__name__}')
        modules_list.append(module.__name__)
        # print(module.__name__)
        # dules_list.append(t.__name__)
        for ax in dir(module):
            # print(ax)
            t = getattr(module, ax)
            if (inspect.ismodule(t)):
                if module.__name__ in t.__name__:
                    q.put(t)
                # else:
                    # print(f'Dont consider {module.__name__}, {ax}, {t.__name__}')
            if (inspect.isclass(t)):
                # Commenting because allowlist should only contain methods
                # allowlist[module.__name__ + '.' + t.__name__] = module.__name__ + '.' + t.__name__
                classes_list.append(module.__name__ + '.' + t.__name__)
            if (inspect.ismethod(t) or inspect.isfunction(t)):
                # print(f't for debug: {t} {module}')
                try:
                    # try block
                    d = typing.get_type_hints(t)
                    if (not d): 
                        empty_typing_hints.append(module.__name__ + '.' + t.__name__)
                    else:
                        if 'return' in d.keys():
                            if isinstance(d['return'], typing._GenericAlias):
                                # print(type(d['return']))
                                # print(get_origin(d['return']))
                                allowlist[module.__name__ + '.' + t.__name__] = get_origin(d['return']).__name__
                            else:
                                # print(d['return'])
                                allowlist[module.__name__ + '.' + t.__name__] = d['return'].__name__
                        else:
                            print(f'No return in keys {t}')

                except Exception as e:
                    print(f'get_type_hints didnt work: {e}')
                
    
    return allowlist, empty_typing_hints, modules_list, classes_list

In [72]:
import xgboost
allowlist,empty_typing_hints, modules_list, classes_list = scraping_lib(xgboost)

get_type_hints didnt work: name 'NDFrame' is not defined
No return in keys <function from_pystr_to_cstr at 0x7fd2ac847dc0>
get_type_hints didnt work: No module named 'dask'
get_type_hints didnt work: No module named 'dask'
get_type_hints didnt work: '_SpecialForm' object has no attribute '__name__'
get_type_hints didnt work: No module named 'dask'
get_type_hints didnt work: No module named 'dask'
get_type_hints didnt work: No module named 'dask'
get_type_hints didnt work: '_SpecialForm' object has no attribute '__name__'
get_type_hints didnt work: No module named 'dask'
get_type_hints didnt work: No module named 'dask'
get_type_hints didnt work: No module named 'dask'
get_type_hints didnt work: No module named 'dask'
get_type_hints didnt work: '_SpecialForm' object has no attribute '__name__'
get_type_hints didnt work: No module named 'dask'
get_type_hints didnt work: name 'NDFrame' is not defined
get_type_hints didnt work: No module named 'dask'
get_type_hints didnt work: No module na

In [73]:
allowlist

{'xgboost.core._array_interface': 'bytes',
 'xgboost.core._get_booster_layer_trees': 'tuple',
 'xgboost.core.find_lib_path': 'list',
 'xgboost.core.from_cstr_to_pystr': 'list',
 'xgboost.dask._assert_dask_support': 'NoneType',
 'xgboost.dask._can_output_df': 'bool',
 'xgboost.dask._cls_predict_proba': 'PredtT',
 'xgboost.dask._create_device_quantile_dmatrix': 'DeviceQuantileDMatrix',
 'xgboost.dask._create_dmatrix': 'DMatrix',
 'xgboost.dask._get_worker_parts': 'list',
 'xgboost.dask._get_worker_parts_ordered': 'list',
 'xgboost.dask._get_workers_from_data': 'list',
 'xgboost.dask._infer_predict_output': 'tuple',
 'xgboost.dask._start_tracker': 'dict',
 'xgboost.dask._unzip': 'list',
 'xgboost.dask._wrap_evaluation_matrices': 'tuple',
 'xgboost.data._array_interface': 'bytes',
 'xgboost.data._transform_np_array': 'ndarray',
 'xgboost.data.dispatch_device_quantile_dmatrix_set_data': 'NoneType',
 'xgboost.libpath.find_lib_path': 'list',
 'xgboost.sklearn._cls_predict_proba': 'PredtT',
 '

In [74]:
package_support = {}

package_support['lib'] = xgboost.__name__
package_support['class'] = classes_list
package_support['modules'] = modules_list
package_support['methods'] = allowlist

In [76]:
import json
# jsonString  = json.dumps(package_support)
with open("package_support.json", "w") as outfile: 
    json.dump(package_support, outfile)

# Current Limitations (to work on next)


1. `get_type_hints didnt work:` type of issues 
2. `No return in keys` type of issues
3. `statsmodels` can't be scrapped using this, hence look for an alternate/ better approach. 
4. Convert to single json 
5. Convert to a de-duplicated better json configuration. 
